In [1]:
!pip install -q ultralytics roboflow==1.* opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 98.1 MB/s eta 0:00:00


In [3]:
import torch
print("GPU disponible:", torch.cuda.is_available())
!nvidia-smi

GPU disponible: True
Fri Aug 29 10:33:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P8             12W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------------

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_KEY")
project = rf.workspace("computervisionfactoria").project("factoria-f5-citpf")
version = project.version(5)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Factoria-F5-5 in yolov8:: 100%|██████████| 9330/9330 [00:01<00:00, 5106.18it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
from ultralytics import YOLO
import os

data_yaml = os.path.join(dataset.location, "data.yaml")
model = YOLO("yolov8m.pt")  # empieza con n (nano); luego puedes probar s o m

results = model.train(
    data=data_yaml,
    epochs=75,
    imgsz=640,
    device=0,          # <--- 0 = usa la primera GPU
    name="factoria_f5_gpu"
)

Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Factoria-F5-5/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=75, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=factoria_f5_gpu, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0,

In [6]:
metrics = model.val(data=data_yaml, split="test", imgsz=640)
print(metrics)

Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,843,234 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 13.4±4.5 MB/s, size: 48.5 KB)
val: Scanning /content/Factoria-F5-5/test/labels... 380 images, 130 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 380/380 876.3it/s 0.4s
val: New cache created: /content/Factoria-F5-5/test/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 7, len(boxes) = 416. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 24/24 2.3it/s 10.3s
                   all        380        416      0.878      0.815      0.866      0.478
                    F5         25         45      0.844      0.489    

In [ ]:

import shutil
import os

folder_name = 'runs'
archive_name = 'runs_backup'


shutil.make_archive(archive_name, 'zip', folder_name)


from google.colab import files
files.download(f'{archive_name}.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
pred = model.predict(
    source=os.path.join(dataset.location, "test", "images"),
    conf=0.25,
    imgsz=640,
    save=True,
    project="runs/predict",
    name="factoria_f5_demo"
)
print("Resultados guardados en:", pred[0].save_dir)


image 1/380 /content/Factoria-F5-5/test/images/1656606217870_jpg.rf.c245a3a36a68ec778e2245fb95191c70.jpg: 640x640 1 F5, 36.2ms
image 2/380 /content/Factoria-F5-5/test/images/1665735788978_jpg.rf.f139739a00abaa2085200076cc285f49.jpg: 640x640 1 Factoria, 36.2ms
image 3/380 /content/Factoria-F5-5/test/images/1665735790387_jpg.rf.4f71964282a1f58be4fb71bbbde9feed.jpg: 640x640 1 F5, 1 Factoria, 36.2ms
image 4/380 /content/Factoria-F5-5/test/images/1667459436404_jpg.rf.f8b7a0d430ffc6066520ded784750e6b.jpg: 640x640 1 Factoria, 36.1ms
image 5/380 /content/Factoria-F5-5/test/images/1682069898124_jpg.rf.63872d2838ea2976ef3017e17257f2e0.jpg: 640x640 2 Factorias, 36.1ms
image 6/380 /content/Factoria-F5-5/test/images/1696235958053_jpg.rf.dd4305de1c2925af5df00cc1e6e27e05.jpg: 640x640 1 Factoria, 28.4ms
image 7/380 /content/Factoria-F5-5/test/images/1737560543054_jpg.rf.5e9f07ac7a707a324c3c58d3cf067c13.jpg: 640x640 1 Factoria, 1 Fundacion Orange, 1 SomosF5, 21.4ms
image 8/380 /content/Factoria-F5-5/t